<a href="https://colab.research.google.com/github/ChaitaliV/Objective-criterias-to-quantify-the-accuracy-of-explanation/blob/main/Explainability%20models/IG_MuRIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install alibi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# If you are can't install captum due to "A UTF-8 locale is required. Got ANSI_X3.4-1968" error rn the below commanda on terminal
#sudo locale-gen en_US.UTF-8
#sudo update-locale LANG=en_US.UTF-8 LC_ALL=en_US.UTF-8
# then restart the rum time and install it
# Install captum using pip
!pip install captum

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import torch
from transformers import BertTokenizer
import matplotlib.pyplot as plt
from transformers import pipeline
from alibi.explainers.integrated_gradients import IntegratedGradients

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model=torch.load(r'/content/drive/MyDrive/Saved Models/MuRIL.pt',map_location=torch.device('cuda'))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('google/muril-base-cased', do_lower_case=True)

In [ ]:
def adapter(data):
  #inputs = tokenizer(data, return_tensors="pt")
  inputs = tokenizer(data, 
          return_tensors='pt', 
          padding=True, 
          truncation=True, 
          max_length=128)
  return inputs

In [ ]:
ig = IntegratedGradients(model.module)

AttributeError: ignored

In [ ]:
text = "This is a sample input text."
X = adapter(text)
explanation = ig.explain(X, target=1)

TypeError: ignored

In [ ]:
## Define Wrpper for making prediction using MuRIL model
class AutoModelWrapper(tf.keras.Model):

  def __init__(self, model_muril, **kwargs):
    super().__init__()
    self.model_muril = model_muril
  
  def call(self, inputs, attention_mask = None):
    out = self.model_muril(inputs,attention_mask = attention_mask)
    return tf.nn.softmax(out.logits)

  def get_config(self):
    return {}

  @classmethod
  def from_config(cls, config):
    return cls(**config)

auto_model = AutoModelWrapper(model)
max_features = 20000
max_len = 20

In [ ]:
# Text sentence for the analysis
z_test_sample = ['I feel a little sad and angry these days']

# Tokenize the sentence and convert it into input tensor
z_test_sample = adapter(z_test_sample)
z_test_sample_input_ids = z_test_sample['input_ids']

# get tensor for model prediction
kwargs = {k: tf.constant(v) for k,v in z_test_sample.items() if k == 'attention_mask'}


In [ ]:
## define the integrated gradient method
n_steps = 20
method = 'gausslegendre'
internal_batch_size = 5
ig = IntegratedGradients(auto_model, n_steps = n_steps, method = method, internal_batch_size = internal_batch_size) # TensorFlow model

In [6]:
import tensorflow as tf
from captum.attr import IntegratedGradients

# Load your TFAutoModelForSequenceClassification model
model = BertForSequenceClassification.from_pretrained('google/muril-uncased-base')

# Define your input text
input_text = "This is a sample input text."

# Convert input text to tokenized input tensor
tokenizer = AutoTokenizer.from_pretrained('google/muril-uncased-base')
tokenized_input = tokenizer.encode_plus(input_text, return_tensors="pt")
input_ids = tokenized_input['input_ids']
attention_mask = tokenized_input['attention_mask']

# Define your prediction function
def predict(input_ids, attention_mask):
    return model(input_ids, attention_mask=attention_mask, training=False).logits

# Instantiate IntegratedGradients
integrated_gradients = IntegratedGradients(predict)

# Convert input_ids and attention_mask to TensorFlow tensors
input_ids = tf.constant(input_ids)
attention_mask = tf.constant(attention_mask)

# Compute integrated gradients
attributions, delta = integrated_gradients.attribute((input_ids, attention_mask), target=1, return_convergence_delta=True)

# Convert the attributions to a NumPy array
attributions_np = attributions.numpy()

# Print the attributions
print("Attributions:", attributions_np)


NameError: ignored

In [13]:
from transformers import BertTokenizer, BertForSequenceClassification
import tensorflow as tf
import numpy as np
from captum.attr import IntegratedGradients

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [17]:
import torch

In [29]:
# Tokenize input text
input_ids = tokenizer("I hate my life, it sucks", 
          return_tensors='pt', 
          )

# Get prediction from BERT model
logits = model(input_ids)[0]
predicted_label = np.argmax(logits.detach().numpy())

# Define the reference input for Integrated Gradients (e.g., an all-zero tensor with same shape as input_ids)
reference_ids = torch.zeros_like(input_ids)

AttributeError: ignored

In [21]:
# Convert input_ids and reference_ids to PyTorch tensors
input_ids = input_ids.squeeze(0)
reference_ids = reference_ids.squeeze(0)

# Define the Integrated Gradients attribution method
ig = IntegratedGradients(model)

In [23]:
inputs = (input_ids,)
references = (reference_ids,)

In [26]:
input_ids

tensor([ 101, 2023, 2003, 1037, 3893, 6251, 1012,  102])

In [27]:
predicted_label

0

In [25]:
# Compute attributions using Integrated Gradients
attributions = ig.attribute(inputs=input_ids, target=predicted_label, n_steps=50)

ValueError: ignored